In [2]:
# Importing all libraries
import pandas as pd
import numpy as np
import math
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
import matplotlib.pyplot as plt

Creating the Class

In [3]:
class LinearRegression:

    def __init__(self, X, y, test_size = 0.3, random_state = 0):
        self.X = X # original X matrix
        self.y = y # original y vector
        # Call a split function to save the splits as attributes
        # Now the class will already contain the splits as attributes
        self.splitdata(test_size, random_state)

    def splitdata(self, test_size, random_state):
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size = test_size, random_state=random_state, shuffle=True)
        self.X_train = X_train # X train split
        self.X_test = X_test # X test split
        self.y_train = y_train # y train split
        self.y_test = y_test # y test split
        print("Splits Created and stored as attributes")
        return
    
    def normalizetrain(self):
        mean = np.mean(self.X_train, axis=0)
        std = np.std(self.X_train, axis = 0)
        self.X_train = (self.X_train-mean)/std # updated X_train
        self.X_mean = mean # mean of X train
        self.X_std = std # std of X train
        return
    
    def normalizetest(self):
        self.X_test = (self.X_test - self.X_mean)/self.X_std # updated X_test
        return
    
    def rank(self, X):
        rank = np.linalg.matrix_rank(X)
        return rank
    
    def checkfullrank(self, X):
        rank = self.rank(X)
        if rank == min(X.shape):
            self.fullrank = True
        else:
            self.fullrank = False
    
    def checklowrank(self):

    



# Add X0 add
# Use X0 add in fit function

Splits Created and stored as attributes
